In [6]:
# Phase 1: Setup and Configuration

# Cell 1: Core Imports and Response Structure
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# Cell 2: Ollama Client Implementation
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Enhanced JSON parsing with fallback"""
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
            
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Model verification with status checks"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Model pulling with progress tracking"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generation with configurable timeout and retries"""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 3: Base Agent Framework
class BaseAgent:
    """Timeout-aware base agent"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        """Generation with time budget tracking"""
        start_time = time.time()
        timeout_occurred = False
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

# Cell 4: Prompt Integration and Saving the Response to a Pickle File

# Define the prompt
prompt = "How can I as a masochistic male generate a sexually satisfying and enjoyable sexual experience for myself with my partner?"

# Initialize the Ollama client and the base agent
client = OllamaClient()
agent = BaseAgent(client)

# Generate the therapeutic response using the prompt
response = agent.safe_generate(prompt)

# Save the response object to a pickle file
with open("therapeutic_response.pkl", "wb") as f:
    pickle.dump(response, f)

print("Response saved to therapeutic_response.pkl")


Response saved to therapeutic_response.pkl


In [ ]:
# Phase 2: Sex-Positive Processing Pipeline

# Cell 5: Desire Analysis Agent
class IntimacyContextAnalyzer(BaseAgent):
    """Analyzes intimacy needs and communication patterns"""
    def analyze_desires(self, input_text: str) -> Dict:
        prompt = f"""Analyze intimacy context (sex-positive focus):
        User Statement: "{input_text[:2000]}"
        
        Identify:
        - Expressed/unexpressed desires
        - Communication style about intimacy
        - Emotional blocks/opportunities
        - Potential exploration pathways
        - Consent awareness indicators
        
        Output JSON with:
        - communication_style: str
        - expressed_desires: List[str]
        - potential_explorations: List[str]
        - communication_improvements: List[str]
        - affirmation_opportunities: List[str]"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

def analyze_desires_from_pickle():
    with open("therapeutic_response.pkl", "rb") as f:
        response = pickle.load(f)
    
    client = OllamaClient()
    analyzer = IntimacyContextAnalyzer(client)
    
    analysis = analyzer.analyze_desires(response.text)
    with open("desire_analysis.pkl", "wb") as f:
        pickle.dump(analysis, f)
    print("Desire analysis saved to desire_analysis.pkl")

analyze_desires_from_pickle()

# Cell 6: Exploration Generator
class IntimacyActionGenerator(BaseAgent):
    """Generates personalized intimacy enhancement actions"""
    def generate_actions(self, analysis: Dict) -> List[Dict]:
        prompt = f"""Create sex-positive action plan:
        Context: {json.dumps(analysis)[:3000]}
        
        Suggest 5-7 actions including:
        - Communication exercises
        - Phrases the partner should say
        - Sensory exploration ideas
        - Exciting Role-play scenarios
        - Connection-building activities
        
        Format as JSON list with:
        - action_type: str
        - description: str
        - purpose: str
        - difficulty: str (beginner/intermediate/advanced)"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

def generate_action_plan():
    with open("desire_analysis.pkl", "rb") as f:
        analysis = pickle.load(f)
    
    client = OllamaClient()
    generator = IntimacyActionGenerator(client)
    
    action_plan = generator.generate_actions(analysis)
    with open("action_plan.pkl", "wb") as f:
        pickle.dump(action_plan, f)
    print("Action plan saved to action_plan.pkl")

generate_action_plan()

# Cell 7: Updated IntimacyCustomizer
class IntimacyCustomizer(BaseAgent):
    """Tailors suggestions to individual preferences"""
    def customize_actions(self, actions: List[Dict], analysis: Dict) -> Dict:
        prompt = f"""Refine intimacy plan:
        Initial Plan: {json.dumps(actions)[:3000]}
        User Context: {json.dumps(analysis)[:2000]}
        
        Format response as JSON with:
        {
            "plan_summary": "brief description",
            "actions": [
                {
                    "action_type": "string",
                    "description": "string",
                    "preparation_steps": ["list"],
                    "ideal_timing": "string",
                    "consent_checkpoints": ["list"]
                }
            ]
        }"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)
def refine_action_plan():
    with open("action_plan.pkl", "rb") as f:
        actions = pickle.load(f)
    with open("desire_analysis.pkl", "rb") as f:
        analysis = pickle.load(f)
    
    client = OllamaClient()
    customizer = IntimacyCustomizer(client)
    
    refined_plan = customizer.customize_actions(actions, analysis)
    with open("refined_plan.pkl", "wb") as f:
        pickle.dump(refined_plan, f)
    print("Refined plan saved to refined_plan.pkl")

refine_action_plan()

Desire analysis saved to desire_analysis.pkl


2025-02-20 14:24:27,665 - WARNING - Generation timed out (attempt 1)
2025-02-20 14:24:27,767 - ERROR - Generation timed out after 300s


In [5]:
# Cell 8: Robust Results Display
def display_results():
    try:
        with open("refined_plan.pkl", "rb") as f:
            plan_data = pickle.load(f)
        
        print("\nPersonalized Intimacy Plan:")
        
        # Ensure we're working with a dictionary
        if isinstance(plan_data, dict):
            actions = plan_data.get('actions', [])
        elif isinstance(plan_data, list):
            actions = plan_data
        else:
            actions = []
        
        # Display first 3 actions safely
        for idx, action in enumerate(actions[:3], 1):
            print(f"\nAction {idx}:")
            print(f"Type: {action.get('action_type', 'Connection Activity')}")
            print(f"Purpose: {action.get('purpose', 'Enhancing intimacy through mutual exploration')}")
            print(f"Steps: {', '.join(action.get('preparation_steps', ['Create comfortable environment']))}")
            
    except Exception as e:
        print(f"Error displaying results: {str(e)}")

if __name__ == "__main__":
    display_results()


Personalized Intimacy Plan:
